# 5. Geth 거래

Last Updated 20200421TUE1000 20190410WEB1530 20180717TUE1700 201801231MON1800 20160207


## 학습내용

### 목표

- 트랜잭션, ABI, 블록체인, 마이닝, gas 학습

### 목차

* 5.1 [메시지](#5.1)
* 5.1.1 call
* 5.1.2 sendTransaction
* 5.2 ABI 명세
* 5.3 트랜잭션 구성
* 5.3.1 비트코인의 트랜잭션
* 5.3.2 트랜잭션
* 5.4 트랜잭션 처리 단계
* 5.5 블록체인
* 5.6 마이닝
* 5.7 gas


### 문제

* 문제 5-1: [다른 계정으로 송금 거래](#5-1)


## 5.1 메시지
<a id='5.1'></a>

이더리움에서의 메시지는 **계정 간**, 예를 들면 **자신의 계정에서 다른 사람의 계정 또는 컨트랙 계정**으로 
전달되는 **데이터**와 **value**를 말한다.

메시지, Message는 우리가 자주 사용하는 핸드폰의 **송금, 입금과 같은 페이 기능**,
금액을 정하고 상대 번호를 지정하고 전송하는 서비스와 비슷하다.
핸드폰에서의 송금 기능이 이더리움과 비교해서 어떻게 다른지 그 메시지를 서로 비교해 보자.

구성 | 핸드폰 | 이더리움 메시지 | 비교
-----|-----|-----|-----
데이터 | 수신측에서 읽을 수 있는 글 | **바이트코드** | 핸드폰 문자는 사람이 읽을 수 있는 텍스트이고 이더리움 메시지는 EVM에서 처리할 수 있는 바이트코드로 작성한 메시지 또는 컨트랙 호출.
value | 송금 | 입금, 송금 등 금액 | 휴대폰 번호로 송금을 해도 '금액'이 실물가치가 있어야 한다. 물론 이더리움에서도 실물가치가 있는 'value' 금액으로 전송해야 한다.
송신 | 전화번호 (또는 계좌번호) | 내, 외부계정 | 휴대폰 번호로 송금을 해도 은행통장과 같은 기능이 있어야 한다. 반면 이더리움에서는 입금 또는 송금 은행통장 없이 가능하다.
수신 | 전화번호  (또는 계좌번호) | 내, 외부계정 | 상동


메시지는 계정의 서명이 필요한지에 따라 1) 메시지 call과 2) 트랜잭션으로 구분한다.
**메시지 call은 서명을 하지 않고**, **트랜잭션은 서명**을 해야 한다.
다시 말하면, 트랜잭션은 전송자에 의해 서명이 된 메시지를 말한다.

함수 | 설명 | 블록체인에 기록
-----|-----|-----
call | 로컬에서 실행, 블록체인에 전송되지 않는다. 따라서 블록체을 변경하지 않고, 읽기만 하는 경우, 비용이 발생하지 않음 | NO
sendTransaction | 외부계정에 의해 발생하고, 사인해서 블록체인에 전송된다. 블록체인을 변경할 경우 사용 | YES 반환값은 hash.

또한 메시지를 보내는 측, **외부계정에서 발생하는지 또는 내부계정에서 발생하는지에 따라 메시지를 구분**할 수 있다.
외부계정, 예를 들어 갑돌이 계정에서 **contract**으로 보낼 수 있다. 이 경우 받는 주소가 블록체인 상의 contract이므로 해당 소스코드가 실행된다. 또는 내부계정에서 **상대방 계정**, 예를 들어 갑순이 계정으로 직접 전송할 수도 있다.


### 5.1.1 call
call() 함수는 로컬 블록체인에서의 호출이라 블록체인에 전송되거나 기록되지 않는다.
따라서 **사인이 필요없고**, gas가 발생하지 않는다.
따라서 네트워크 참여자가 그 함수가 호출되었는지 알지 못한다.
call() 함수는 실행되고 나서 그 반환 값을 가질 수 있다.
web3.js API는 **web3.eth.call**이다.

내부 계정, 즉 contract은 다른 contract이나 계정으로 메시지 call을 할 수 있다.
그 예는:

```python
call.gas(0).value(number)()
```

> Solidity Documentation의 메시지 호출 설명

> 컨트랙은 다른 **컨트랙**을 호출하거나, **컨트랙이 아닌 다른 계정**으로 메시지 호출이라는 수단을 통해 Ether를 전송할 수 있다.
메시지 호출은 트랜잭션과 비슷한데, 그 이유로는 송신, 수신, 데이터, Ether, gas, 반환 값을 가진다는 점이다.

### 5.1.2 sendTransaction

sendTransaction() 함수는 블록체인에 전송되고 기록된다.
**사인**을 해야 하고, gas가 필요하다.
트랜잭션은 **반환 값이 없고, 그 결과 값은 해시 값**이다.
프로그래밍을 할 때 반환 값은 함수가 올바르게 실행이 되었는지를 테스트할 때 유용하게 사용되는데, 블록체인에서는 이러한 반환값이 없다는 것은 매우 불펴한게 느껴진다. **반환 값을 알려면 event를 발생**시켜서 로그에 적어주고, 나중에 그 로그를 확인해야 한다.
해시 값이 주어지면, 트랙잭션은 마이닝이 되어야 하고 반환 값은 없다.
web3.js API는 **web3.eth.sendTransaction**이다.

```python
coin.send(web3.eth.accounts[0], 100,{from:web3.eth.accounts[0],gas:100000});
coin.send.sendTransaction(web3.eth.accounts[0], 100,{from:web3.eth.accounts[0],gas:100000});
```

> Solidity Documentation의 트랜잭션 설명

> 트랜잭션이 메시지인지 구분이 모호할 수 있다. 공식 문서를 참고하면 도움이 되겠다.
> 트랜잭션은 한 계정에서 다른 계정으로 전송되는 메시지이다. 바이너리 데이터와 Ether를 포함할 수 있다.
원문을 읽어보면 "A transaction is a message that is sent from one account to another account. It can include binary data (which is called "payload") and Ether"


## 5.2 ABI 명세

Application Binary Interface (ABI)는 컨트랙을 호출하기 위해 사용하는 방식이다.
블록체인 외부에서나 또는 블록체인 내부에서 **컨트랙 서로 호출할 때, 함수 및 인자를 ABI 표준에 따라 부호화**해서 호출하게 된다.
이 표준은 곧 살펴보겠지만, 사람 눈으로 그 부호화되는 과정을 이해하기는 어렵다. 그러나 걱정하지는 말자. 이더리움 내부에서 알아서 하므로 개발자가 직접 이 방식으로 하지 않아도 된다.

예를 들어, Foo의 baz()함수를 호출한다고 하자.
우선 함수와 인자를 각 각 인코딩하고 조합해서 16진수 데이터로 만들어 호출하게 된다.
출처: https://solidity.readthedocs.io/en/develop/abi-spec.html
```python
contract Foo {
  function bar(fixed[2] xy) {}
  function baz(uint32 x, bool y) returns (bool r) { r = x > 32 || y; }
  function sam(bytes name, bool z, uint[] data) {}
}
```

1) **함수명 부호화**

호출할 **함수시그니처** (함수명과 인자, 이때 인자는 **컴마로 분리하고 공백 없이** 적는다)의 **sha3 해시값**으로 만들고, **앞 4 바이트**를 선택한다. 이를 **function selector**라고 한다.
함수 시그니처 ```baz(uint32,bool)```의 Keccak hash ASCII 앞 4바이트를 선택한다.

```python
<address>.call(bytes4(bytes32(sha3("baz(uint32,bool)")))
```

```python
> web3.sha3('baz(uint32,bool)').substring(0,10);
"0xcdcd77c0"
```

2) **인자 값을 부호화**

**32 바이트**로 만들기 위해 **앞에 0으로 채운다**.
인자가 69일 경우, 앞은 0으로 채워서 32바이트로 만든다.
```python
> web3.toHex(69)
"0x45"
0x0000000000000000000000000000000000000000000000000000000000000045 - 인자 uint32 69 (32바이트)
0x0000000000000000000000000000000000000000000000000000000000000001 - 인자 bool 1  (32바이트)
```

3) **데이터 조합**

위 1) function selector부호값과 2) 인자부호값를 합친다. 0x를 제거한다.
아래에서 볼 때 몇 줄로 나누어져 있지만 그것은 공간상의 제약때문에 그러하고, 한 줄로 쭉 연결이 된 문자열이된다.

```python
0xcdcd77c00000000000000000000000000000000000000000000000000000000000000045
0000000000000000000000000000000000000000000000000000000000000001
```

4) **함수 호출**

함수를 호출할 때 ```data``` 항목에 위와 같이 생성된 ABI 부호를 적는다.

```python
web3.eth.sendTransaction({
    from: eth.accounts[0],
    to: "0x672807a8c0f72a52d759942e86cfe33264e73934",
    data: "0xcdcd77c000000000000000000000000000000000000000000000000000000000000000450000000000000000000000000000000000000000000000000000000000000001",
    gas: 400000}
)
```

## 5.3 트랜잭션 구성

### 5.3.1 비트코인의 트랜잭션

비트코인에서의 거래는 한 주소에서 다른 주소로 금액이 이체되는 것으로, 네트워크에 전파되고 모여서 블록으로 만들어지게 된다.
거래는 아래 표에서 보듯이 버전, locktime 타임스탬프, 입금, 출금 관련 항목으로 구성한다.


길이 | 항목 | 설명
-----|-----|-----
4 bytes | Version | 버전정보
1-9 bytes | Input counter | in 개수
variable | in | array로서 이전 거래의 out에서 가져와 본 거래의 out으로 지급된다.사용하게 되는 잔액을 가지고 있는 transactionHash, script를 가진다.
1-9 bytes | output counter | out 개수
variable | out | array로서 in보다 같거나, 적어야 한다 (적으면 처리비용으로). UTXO OpCodes: Scripts - 6...4사용하고 남는 잔액들 unspent transaction output UTXO. script, value를 가진다.
4 bytes | locktime | unix timestamp 또는 블록번호

아래 사례에서 보듯이, 입출금이 하나가 아니고 복수일 수 있다는 점에 주의한다.
거래에서 발생한 미사용 잔액이 수합되어야 하는 경우 input이 여러 개가 된다.
```ins```는 이전 거래 ```outpoint``` 2건에서 미사용 잔액을 수합하여 ```output```으로 지급한다.
출금 또한 수신자가 1명이라도, 그 잔액이 자신에게 돌려지므로 **최소 2개**가 된다.
하나는 수신처로 송금되는 금액, 다른 하나는 거스름돈에 해당하는 잔액이다.
```Value```는 Satoshi 금액이다 (1 BTC = 100,000,000 Satoshi).

In [2]:
import bitcoin, pprint
tx='0100000002349cbebe5628707553812eb0e591dc49047d743e836006773a65b55cf4444a5a030000006a47304402203f02d209168bb9c4e8fab3b2f2aca19be1f9032eaa85be3a029641eecf69e60d022055d7e22914bb4741ee8c1ab73e23766de034a82dae22dad0c13684d6cb098b9e012102c3a7056ad278bf04f2e5546e29466d8175ec4a052ad8f9901b32d53e442545a8ffffffff0890e18ae1953971b7b64ddedcb75c034e5c16969e498358b517960fd0186238010000006b4830450221009511497e136765037a75f75a126168502bfee73a7c286ea775968a7140368ef002200cfc776ade17b9444bb7ad13e71852974befd1d6fcfa196df677cd72a697c63b012102b8963213667c71471d62361fe27aad7ab2b8b102a6d93f7367634466e04b5422ffffffff022085d0020000000017a914ed91639c578b4c1f526525e6dcbc9fbc505fe7698751bd0d00000000001976a9145f9a3e6e8029dd934a2a00a17aa8254c0975219988ac00000000'
tx_structure = bitcoin.deserialize(tx)
pprint.pprint(tx_structure)

{'ins': [{'outpoint': {'hash': '5a4a44f45cb5653a770660833e747d0449dc91e5b02e815375702856bebe9c34',
                       'index': 3},
          'script': '47304402203f02d209168bb9c4e8fab3b2f2aca19be1f9032eaa85be3a029641eecf69e60d022055d7e22914bb4741ee8c1ab73e23766de034a82dae22dad0c13684d6cb098b9e012102c3a7056ad278bf04f2e5546e29466d8175ec4a052ad8f9901b32d53e442545a8',
          'sequence': 4294967295},
         {'outpoint': {'hash': '386218d00f9617b55883499e96165c4e035cb7dcde4db6b7713995e18ae19008',
                       'index': 1},
          'script': '4830450221009511497e136765037a75f75a126168502bfee73a7c286ea775968a7140368ef002200cfc776ade17b9444bb7ad13e71852974befd1d6fcfa196df677cd72a697c63b012102b8963213667c71471d62361fe27aad7ab2b8b102a6d93f7367634466e04b5422',
          'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': 'a914ed91639c578b4c1f526525e6dcbc9fbc505fe76987',
           'value': 47220000},
          {'script': '76a9145f9a3e6e8029dd934a2a00a17aa8254c0975219

### 5.3.2 트랜잭션

#### 거래 속성

다음에서 보게 되겠지만, 이더리움에서는 미사용잔액을 수합해서 사용하는 방식이 아니라, 입출금 관련항목이 불필요하고
입금주소, 출금주소와 금액으로 단순해지게 된다.
거래는 필요한 속성이 채워져야 한다.

속성 | 바이트 | 설명
----|-----|-----
from | 20 바이트 | 전송 주소. 명시하지 않으면 web3.eth.defaultAccount
to | 20 바이트 | (optional) 수신 주소. 컨트랙 생성하는 경우는 당연히 명시하지 않아도 된다.
value | 32 바이트 | (optional) 전송 Wei 금액
gasLimit | 32 바이트 | (optional, default: To-Be-Determined) 거래처리에 허용되는 최대 개스량
gasPrice | 32 바이트 | (optional, default: To-Be-Determined) 거래 처리에 필요한 개스량 (Wei)
data | 제한 없슴 | (optional)  메시지 데이터 (바이트)
nonce | 32 바이트 | (optional) 계정에서 발생한 거래 순서 번호. 동일한 번호가 있으면 하나는 처리 거절, 또는 순서대로 처리 (즉 뒷 순서가 먼저 처리되고 앞 번호를 취소하는 이중거래 편법을 막음)
v, r, s | 1, 32, 32 바이트 | v, r, s는 거래의 signature에서 구하고, 공개키를 회복하기 위해 사용된다.사인에서 구할 수 있다. v는 공개키를 회복하기 위해 필요한 recovery id (27 또는 28을 가진다. 0, 1 또는 2 값을 가질 수도 있다), (r,s)에서 공개키를 회복하는 방법이 2가지가 있고, 그 중 하나를 식별하기 위해 사용된다. (r은 R의 is computed as the X coordinate of a point R, modulo the curve order n.)

예를 들면:

```python
transactionObject = {
  from: "0xA0A0A0A01",
  to: "0xA0A0A0A02",
  value: web3.eth.getBalance("0xA0A0A0A01"),
  gas: "----",
  gasPrice: "----",
  data: "----",
  nonce: 0,
}
```

## 5.4 트랜잭션의 처리 단계

거래가 발생하면, 마이너에 의해 인증해야 한다.

갑이 을에게 송금 거래를 한다고 하자.
갑은 거래에 디지털서명을 하고
그 거래는 참여 노드들에게 전파된다.
노드들은 거래를 받으면:
- 거래의 **디지털서명**을 인증하여 원본이 맞는지, 소유주의 서명이 맞는지?
- 전송자가 **gas를 지불할 잔고**를 가지고 있는지
- gas가 **gas limit**을 초과하지 않는지 기본적인 검증을 하고 거래를 다시 네트워크에 전파한다.

마이너 노드들은 거래를 블록으로 만들어 가고, 만들면 모든 네트워크참여자에게 공지한다.
블록이 체인에 연결, 블록해시를 다음 블록에 참조하면서 연결이되고 이를 받아들인 것으로 합의한다.

순서 | 단계 | 설명
-----|-----|-----
1 | 거래 생성 | 거래에 **디지털 서명을**하고, 전송하기 위해 바이트 단위로 변환, **직렬화 serialize**한다.
2 | 거래 전송 | 거래가 p2p 네트워크로 전송되고, 노드들 간에 서로 전파된다 (broadcast).
3 | 블럭 생성 | 일정 시간 동안 거래를 묶어서 블록이 될 때까지 **pendingTransaction**에 넣는다. 그러나 아직 블록체인에 연결되기 전이므로 후보 블록 **candidate block**이 된다. 마이너들이 이렇게 묶은 블록 후보는 네트워크로 전파된다. 마이너는 자신들이 거래를 수집하여 블록으로 만들기 때문에 서로 같지 않을 수 있다. 거래가 도착 순서대로 묶어지는 것이 아니라, **우선순위**가 높은 거래부터 시작해서 블록으로 만들어진다 (이더리움에서는 처리비가 높은 거래가 우선순위가 높다. 비트코인에서는 잔고UTXO의 age, 금액이 클수록 높다, Sum (Value of input x Input Age) / Transaction Size). 따라서 부모hash가 없는 **고아 거래orphan transactions**이 있을 수 있다. 포함이 안된 거래는 기다려서 다음 블록에 추가된다.
4 | 블록인증 | 앞서 만들어진 블록을 인증하게 된다. 마이닝은 Proof of Work, 즉 **hash puzzle을 풀어 nonce를 정하는 작업**이다. 블록 해시를 가장 먼저 찾아낸 노드는 이를 네트워크에 알리고, 다른 노드들은 새로 생성된 블록의 유효성을 검사하고 인증한다. **가장 긴 길이를 가진 체인을 선택**하여 (고아 블록은 짧은 길이를 가지게 되므로 선택되지 않는다), 블록을 앞 블록에 **전의 블록 hash를 찾아 체이닝**되어 블록에 추가된다. 한다. 참여한 노드들 중에서 가장 먼저 해시값을 찾아낸 노드가 보상을 받는다. 2종류의 보상: 1) 새롭게 생성된 코인을 획득 (2015년 25비트코인 생성), 2) 거래비용. 2016개마다 문제의 난이도를 조정한다.
5 | 새로운 블록 전파 | 새로운 블럭이 공지되면 **다른 마이너들은 이 블록 해시값이 올바른지 검사하고 받아들이게 되면서 마이닝은 종료**하게 된다. 거래기록은 중앙 서버가 가지고 있는 것이 아니라, 분산 네트워크에 실시간 공유된다. 공유되고 나면 수정할 수 없다. 로컬 블럭체인 동기화, 마이닝하지 않은 경우 동기화가 뒤떨어질 수 있다.

## 5.5 블록체인

### 블록헤더

블록체인은 블록의 연결로 헤더와 바디로 구분된다.
블록헤더는:
1) 프로토콜 버전(Version), 2) 이전 블록 해시 값 (Previous Block Hash), 3) 채굴속도를 조정하는 난이도 (Difficulty), 4) 블록이 생성된 시간 (Timestamp), 5) Target Hash를 맞추기 위해 사용하는 난스(Nonce), 6) 각 블록의 해시 값을 결합하여 계산하는 머클 루트의 해시값 (Merkle Root)으로 구성한다. 가벼운 클라이언트의 spv (simple payment verification)는 이러한 블록헤더를 가지게 된다. spv는 거래를 검증하기 위해서는 full node의 머클트리에 요청하여 거래가 블록에 포함되었는지 확인하게 된다.

블록헤더 | 바이트 수 | 설명
-----|-----|-----
version | 4 | 프로토콜 버전
Previous Block Hash | 32 | 이전 블록 해시 값으로 서로 체인으로 연결.
Merkle Root | 32 | 블록에 있는 모든 거래 hash의 root. 따라서 어떤 블럭 해시 값이 변경되면 Merkle Root Hash값도 달라지게 되어 수정이 불가능함.
Timestamp | 4 | 블록 생성 시간
difficulty target | 4 | 난이도를 조정하여 블록생성시간을 유지.
nonce | 4 | Target Hash를 맞추어 나갈 때 조정

블록바디는 발생한 트랙잭션을 저장한다.

### 블록체인

블록체인은 블록의 체인이고, 블록은 **거래를 묶어 이진트리 merkle tree로 구성**해서 가진다.
Merkle tree는 **hash로 이름 붙여진 가지노드 lead node로 구성된 트리**를 말한다.
예를 들어, 트랜잭션 A, B, C를 머클트리로 구성한다면 각 거래 데이터를 해싱해서 나온 결과 값이 각 가지 노드에 저장된다. 부모 노드를 만들기 위해서 자식노드의 32바이트 크기의 트랜잭션 A와 B의 해시 값을 서로 연결해서 64바이트 문자열을 만들고 문자열은 **이중 해시** 처리되어 부모노드의 해시를 생성한다. 
이러한 방식으로 상위에 노드가 하나 남을 때까지 계속 계산하고, 각각의 트랜잭션을 이진트리 형태로 만들 경우 가장 최종적으로 남는 해시 값이 머클트리 루트 해시 값이 된다.

블록 내에서 머클 해시 값을 통해서 **트랜잭션의 무결성**을 검증 할 수 있고, 블록 해시 값을 통해서 **해더 값에 대한 무결성**을 검증할 수 있다.
거래 내역의 위변조를 막기 위해서 해시로 만들고 이것을 트리 형태로 만든 것이다.
어느 한 **블록에 변조가 발생하면, merkle tree로 root까지 전달되어 블록체인 전체가 변조**된다.
이전에 생성된 블록해시 값은 다음 블록 헤더의 'Previous Block Hash' 항목에 저장된다. 각 블록에 트랜잭션이 지속적으로 쌓이면 새롭게 블록을 생성하여 **블록해시 값을 연결고리로 블록을 연결하여 블록체인이 생성**된다.
블록 뒤에 다른 블록이 추가될 때마다, 이를 **confirmation**이라고 하는데 confirmation이 늘어날 수록 수정이 힘들어 진다. 
**fork**는 **같은 블록높이를 가지는 블록이 여러 개**, 부모가 여러 자식 블록을 가지는 경우 발생한다. 복수의 마이너가 거의 동시에 블록을 생산하는 경우이다.
매 **2016블록마다 difficulty가 조정**된다.

> 블록높이 block height

> 블록체인에서 선행블록의 수를 말한다. 예를 들어 genesis block의 블록높이는 0이다.

block header를 생성해보자 (source: bitcoinwiki https://en.bitcoin.it/wiki/Block_hashing_algorithm)

버전, 이전 블록 해시 값, 머클루트 값, 타임스탬프, 난이도, 난스의 hex 값을 이중 해싱해서 생성한다.

In [7]:
import hashlib

In [27]:
version="01000000";  #4 bytes
prevBlockHash="81cd02ab7e569e8bcd9317e2fe99f2de44d49ab2b8851ba4a308000000000000"; #32 bytes
merkleRoot="e320b6c2fffc8d750423db8b1eb942ae710e951ed797f7affc8892b0f1fc122b";  #32 bytes
timestamp="c7f5d74d"; #4 bytes
difficulty="f2b9441a"; #4 bytes
nonce="42a14695";  #4 bytes
header_hex = version + prevBlockHash + merkleRoot + timestamp + difficulty + nonce
#header_bin = header_hex.decode('hex')  # changes to binary string

header값을 binary로 변환하고, sha256 더블해싱을 한후 hex코드로 출력해보자.

In [54]:
header_bin = header_hex.encode('utf-8')  # changes to binary string
hash = hashlib.sha256(hashlib.sha256(header_bin).digest()).digest()

In [55]:
#hash.encode('hex_codec')    # equals to "encode('hex')"
hash.hex()

'6500f13bc254c59e9f3d77bd0b1999e686fadf7765ae2b59266d1d835b869083'

문자열을 뒤집어 출력해보자. 2자리, 즉 1바이트씩 거꾸로 복사를 하는데, 이를 byte-swap이라고 한다.

In [57]:
hashhex=hash.hex()

In [58]:
#hash[::-1].encode('hex_codec')  # [::-1] all elements in the array reversed
"".join(reversed([hashhex[i:i+2] for i in range(0, len(hashhex), 2)]))

'8390865b831d6d26592bae6577dffa86e699190bbd773d9f9ec554c23bf10065'

### Merkle proof

Merkle tree는 거래를 트리구조로 연결하여 가지고 있다.
최정점의 머클루트는 가지들을 합쳐서 결국 하나의 해시 값을 만드는 것이다. 가지 노드를 하나씩, 예를 들어 TxA와 TxB를 합쳐서 해시값을 구하고, 다른 노드에 대해서도 이 작업을 반복하게 된다. 최하층 노드에 대해 두 개씩 결합이 끝나면, 이런 식으로 그 다음 계층으로 올라가 반복해서 해시값을 구한다.

이렇게 해시로 연결해 놓으면 어떤 장점이 있을까?
특정 transaction이 수정되면, 그 해시가 변경되게 되고, 연쇄적으로 전체 해시가 다르게 되기 때문에 위변조가 거의 불가능하게 된다.

```python
              ---------------
              | Merkle Root |
              ---------------
                     /      byte swap
                    /     double SHA256
                   /   Hash 1 2 + Hash 3 4
             --------------
             |  Hash 1 2  |
             --------------
           /  double SHA256 \
          /    hash1+hash2   \
     byte swap            byte swap
  --------------        --------------
  |   Hash 1   |        |   Hash 2   |
  --------------        --------------
        |                    |
        | SHA256             | SHA256  
Transaction 1 Data   Transaction 2 Data
```

아래 Merkle tree에 거래 TxA, 거래 TxB가 있다고 하고,
여기서는 문자열만으로 구성하고, 이들을 묶어 머클루트를 구해보자.
거래 TxA, TxB를 해싱하고, 바이트 스왑을 한다.
그리고 합쳐서 더블 해싱을 하여 상위 해시 값을 구한다.

각 문자열을 해싱해서 인코딩부터 해준다.
Python 3.0부터는 문자열은 유니코드로 저장된다..
```encode()``` 함수는 유니코드 문자열을 바이트로 변환, 즉 인코딩을 한다.
물론 거래는 문자열만으로 구성되지 않는다.
우선 hashing을 해준다.

In [1]:
import hashlib

nonce=1
txA = 'Hello'.encode()
txB = 'How are you?'.encode()
txC = 'This is Thursday'.encode()
txD = 'Happy new Year'.encode()

In [4]:
hashA=hashlib.sha256(txA)
hashB=hashlib.sha256(txB)

```digest()```는 바이트 형식으로, 반면에 ```hexdigest()```는 16진수로 출력한다.

In [5]:
hashA.hexdigest()

'185f8db32271fe25f561a6fc938b2e264306ec304eda518007d1764826381969'

#### byteswap

byte swap으로 big-endian을 little-endian으로 변환한다.
이런 변환은 Endianness의 차이때문에 한다.
이더리움 EVM은 big endian (예: 0x1234일 때, 왼쪽에서 오른쪽으로0x12 0x34), CPU는 small endian (0x34, 0x12)으로 처리한다.

1 바이트, 즉 2 nibbles씩 출력해보자. 
전체글자수 (즉 32바이트의 2배)에 대해 1 바이트인 2글자씩 반복을 하면서 출력한다.

In [20]:
for i in range(0, hashA.digest_size*2, 2):
    print(hashA.hexdigest()[i:i+2])

18
5f
8d
b3
22
71
fe
25
f5
61
a6
fc
93
8b
2e
26
43
06
ec
30
4e
da
51
80
07
d1
76
48
26
38
19
69


반복문을 pythonic하게 줄여볼 수 있다. 그리고 2바이트씩 잘라서 ```reversed()``` 함수로 뒤집는다.

In [6]:
hashAswap="".join(reversed([hashA.hexdigest()[i:i+2] for i in range(0, hashA.digest_size*2, 2)]))

In [7]:
hashBswap="".join(reversed([hashB.hexdigest()[i:i+2] for i in range(0, hashB.digest_size*2, 2)]))

In [8]:
print("hashAswap: ", hashAswap, "\nhashBswap: ",hashBswap)

hashAswap:  691938264876d1078051da4e30ec0643262e8b93fca661f525fe7122b38d5f18 
hashBswap:  890dfa0aaa970381d9e11e1553ac7ec9ceb53b782c1c2e692bed0614fc7d28df


#### double hashing

두 해시값을 결합한다.

In [9]:
#hashAB=hashA.hexdigest()+hashA.hexdigest()
hashAB=hashAswap+hashBswap

double hashing은 해시 값에 대한 'length extension attack'을 막기 위해 사용한다.

In [11]:
hashlib.sha256(hashlib.sha256(hashAB.encode()).hexdigest().encode()).hexdigest()

'006c195035200dd4ee937e56d7de9c1d2ce4fe1adc0f1e2451f27fc34520546c'

## 5.6 마이닝

마이닝은 거래를 인증하는 과정으로, hash puzzle을 풀고 그 보상으로 새로운 코인을 생성하게 된다.
발생한 모든 거래가 추가되는 것은 아니다.
**목표 값 target hash를 찾아내는 계산**을 하고, 맞추는 경우에 인증되고 블록체인에 비로서 추가된다.
정답을 맞추는 작업을 하는 참여자들을 **마이너**라고 하며, **보상은 정답을 맞춘 최초의 경우**에만 해당 마이너에게 주어진다.
이와 같이 일정한 량의 계산에 따라 거래가 인증되고, 블록체인에 추가될 수 있으므로 이러한 **계산작업증명을 Proof of Work** 이라고 한다.

비트코인의 경우 블록생성시간은 10분으로 설정되어 있으므로 하루 144 블록, 일주 1008블록이 생성된다. 
이더리움은 문제의 난이도는 12초에 1건씩 풀도록 정한다.

> Orphan, Stale

> * **Orphan**  블록체인에서 **부모가 없는 블록**을 말한다. 부모로 삼으려 했던 블록이 인증이 실패하면서 부모가 없어지는 경우 발생한다.
* **stale** 블록은 **자식이 없는 블록**을 말한다. 마이닝은 되었지만, 블록높이가 동일한 다른 블록이 먼저 자식이 생기면서 정상적인 블록체인이 되는 경우 발생한다.

이더리움에서의 합의 알고리즘은 GHOST (Greedy Heaviest Observed Subtree, Zohar and Sompolinsky in December 2013)이다. 가장 무거운 트리가 블록체인에 연결되는 방식이다. 열매가 많이 매달려서 무거워진 나무의 가지를 연상하면 된다. 그 가지를 선택한다는 것이다. 비트코인은 가장 긴 블록을, 반면에 이더리움은 가장 무거운 블록을 추가한다.

> Hash Rate

> Hash Rate는 1초에 Hash를 몇 회하는지 (Number of hashes per second, H/s)
마이닝하는 속도를 말한다. 성능이 좋은 컴퓨터는 당연히 Hash Rate이 높게 마련이다.
H/s는 일반 척도와 같은 단위를 사용하여 나타낼 수 있다.
비트코인이 등장한 초기에는 보통 컴퓨터로 마이닝을 충분히 해낼 수 있었다.
채굴의 난이도가 높아지면서 특별히 마이닝을 하기 위한 **ASIC (Application Specific Integrated Circuit)** 칩을 사용한 고가의 컴퓨터가 사용되고 있다.

구분 | 단위 | 해시 회수
-----|-----|-----
Kilohashes | KH/s | 1000
Megahashes | MH/s | 1,000,000
Gigahashes | GH/s | 1,000,000,000
Terahashes | TH/s | 1,000,000,000,000
Petahashes | PH/s | 1,000,000,000,000,000


> **Ethash**

> 이더리움이 구현한 PoW 알고리즘이다.
이전에 사용하였던 Dagger-Hashimoto 알고리즘을 개선한 버전이다.
Ethash에서는 "sha3_256", "sha3_512" 해시함수가 사용된다.

> Ethash는 Ethash DAG라고 하는 1 GB 데이터 분량을 사용한다. 전체를 가지고 있을 필요가 없는 light client는 16 MB 캐시를 사용한다.
이 데이터는 매 **30,000 블록마다 재생성**되는데, 이 시간을 **epoch**이라고 한다.
마이닝할 때 DAG의 일부를 가져다가 블럭해시와 nonce를 사용해 target hash를 맞춰 나갈 때 사용한다.

> 마이닝을 하기 전에 보상이 지급되는 계정 coinbase를 정해야 한다.
coinbase를 충전할 계정으로 변경하고, reward가 주어질 계정으로 마이닝을 시작한다.
**처음 채굴을 시작하면, 기가규모의 블럭체인을 내려받아야 한다. 약 10분 정도 소요되며, DAG가 구성**된다.
>```python
> miner.setEtherbase(eth.accounts[2])
true
> eth.coinbase
"0x53cbba17cf9bd0735855809bdcb88e232de96f32"
> miner.start();
```

### 난이도

**hash puzzle**은 **block hash**값을 결정하기 위한 문제이다.
식으로 표현하면, **```sha256(sha256(data+nonce)) < dificulty가 반영된 목표 해시값```**이고, 이 식이 충족되도록 nonce를 찾아야 한다.
즉, 블록헤더의 Hash값이 난이도 목표에 제시된 값보다 작은값이 나오게 하는 Nonce값을 찾는 것이다.

예를 들어, **10분에 문제를 풀기를 기대했는데 5분에 풀었다**. 그러면 **난이도는 $\frac{10}{5}$, 2**가 된다.
그러면 **새로운 난이도 = 현재 난이도 x 2**이 된다. 즉 난이도가 2배로 증가하게 된다.
난이도가 1보다 크게 되면 새로운 난이도는 증가하고, 반대는 감소하게 되는 방식이다.

**새로운 목표값 = 이전 목표값 / 난이도**
즉, 난이도가 높아지면 새로운 목표 값이 낮아지고 맞추기가 더 어려워지게 된다.

예를 들어:
* 1부터 10 범위에 들어가는 수를 생성하는데 1분이 걸린다고 하자.
* 목표 값을 5로 정하고, 그 이내의 값이 나오려면 $60s \times \frac{10}{5}$ 즉 2분이 걸린다 (1~5의 사이므로 2배의 시간)
* 목표 값을 3으로 정하면, $60s \times \frac{10}{3}$ 즉 3분 20초가 걸린다.

즉:
* 목표 값이 최대값에 가까우면 쉬워진다. 그 보다 작은 수를 찾는 것은 당연히 쉽게 된다. 예를 들어 $2^{255}-2$ (**최대 32바이트 hash값은 $2^{255}-1$**, 이 보다 하나 적은 블록 hash값을 찾는 것은 쉽다.
* 최대 값과 목표 값의 간극이 벌어질수록 어려워진다.
* 비트코인에서 difficulty는 2016개의 평균시간이 1,209,600 seconds (two weeks)되도록 정한다.

그 절차를 좀 더 자세하게 설명하면:

(1) 거래를 수집하여 블록을 만든다 (비트코인은 1MB 블록)
data=Merkle Root (블록에 있는 모든 거래의 hash 합계) + previous block hash

(2) 블록헤더 (Version + Previous Block Hash + Merkle Root + Timestamp + Difficulty Bits + Nonce) 값을 SHA-256 해시하고 또 재해시를 하여 해시값을 계산한다.

(3) 2에서 계산된 해시값을 Target hash와 비교한다. target hash보다 적으면 정답, 아니면 2번으로 돌아가 nonce값을 증가함.

(4) 문제를 푼 경우, 
문제를 풀면 블럭체인에 승인된 블럭을 맨 뒤에 첨부하고 모든 peer에게 공지.
참여자들에게 공지하고 참여자들로 하여금 계산을 하게 하여 검증되면 합의한다.

(5) 참여자가 많아져 계산속도가 빨라질 수 있지만 설정된 블럭생성시간에 맞추어 난이도가 조정된다. 파이썬에서 ```//```는 정수 나누기 연산이다.
```python
block_diff = parent_diff + parent_diff // 2048 * 
max(1 - (block_timestamp - parent_timestamp) // 10, -99) + int(2**((block.number // 100000) - 2))
```

즉, 부모블록과 현재블록의 생성시간의 차이에 따라 난이도 변경이 이루어진다.  ```parent_diff // 2048```는 난이도 변경범위 (bound divisor of the difficulty)이다.
- **10초 이하**: 난이도 **상향**, ```parent_diff // 2048 * 1``` 만큼 상향
- **10~19초**: 난이도 변동 없슴.
- **20초 이상**: 난이도 **하향** (timestamp 차이에 따라)최소 ```parent_diff // 2048 * -1```에서 최대 ```parent_diff // 2048 * -99```

(6) 블록이 한꺼번에 2개 마이닝 되는 경우, 
두 마이너가 동시에 풀면 fork가 발생
난이도가 높은 블록이 인증받게 된다.
```python
loop
    if sha256(sha256(data+nonce)) < difficulty
        stop
    else change the nonce
```

블록헤더 데이터의 해시 값에 NONCE를 증가시키면서
앞 자리의 0의 개수를 맞출 때까지 반복한다.
찾고자 하는 해시가 ```00000```로 시작한다고 하자.
그러면 최대값은 ```000000FFFF...```가 되겠다
(공간제약으로 6자리 0과 26자리 F를 모두 표현하지 않았다).
즉 그 해시의 최대 값보다 작은 범위 내에 들게 된다.
그러면 멈추고, 그 값을 해시로 정하게 된다.

- NONCE:  100 a6e35d11b8f264ab6ebdf67081b66c1539f11b33fcefd16ca4595549d23daeed
- NONCE:  200 0c5c39f89e19827e3fe915932c2ff1c4c0b3df2275f8598fe07e07e9c1c1c4a2
- NONCE:  300 88fe56b50fa54b4ba5d9ec7ee52ab8a0cf672ec05d1a764b9201d92d6fa75893
- NONCE:  400 650eaeb2cdebbcdd681400dcda4f2463d1d9d7706d8ea1e560deb1dd7a1a35cf
- NONCE:  500 eb7f829ae96f868405aa87b6e74cb100669803d68b0a2c9df02927d332676fc4
...


In [ ]:
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:4]=='00000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

NONCE:  10000000 660b9e057377381579f5c54347901cf462fce656c069a4d0f26bdd4cf1e05e66
NONCE:  20000000 c31d3ddd7bb92312bcc2a88263d92e94c6d17a641ac9e59ac262c775c7f58925
NONCE:  30000000 236077716f4ce428ec83f12bf74fb7ab76450bad60bc3f21d924e200c39a9fa4
NONCE:  40000000 5b07929bd7088f11d6caf0e65a6f44ab3b7f82d32900430cc7c7244b2917e04d
NONCE:  50000000 d9af343cc488cb44b57c30a071033fb4c9fc836f16e6c619be66a26c18b1a31a
NONCE:  60000000 eee30c7b9952680cc2cf51a093586447c227fe78f3b4652684f2e5d8e589d4d9
NONCE:  70000000 3d2125955e4fc1a9a154bafa910efb11cd265c97e6400341811c74fa2b09c0c1
NONCE:  80000000 3c32aeb8681a6041e26befb7e060b74d992e00ac0c8ef684b641dc0ea4fa30dd
NONCE:  90000000 a3f699e61d3182cffe35ffdb5f354b04ccbe7280474eb30dd76ea0fc3f037c88
NONCE:  100000000 e196aa710ba089a8eb23bbfe5052183dc6f1c1f57733b1f614ce1e0daf8bff9b
NONCE:  110000000 a047c1fd03a26cc8489fd24fb99cef4a1d0b87ecda1e41daaf6e57066f540d60
NONCE:  120000000 45edd32c46085f0901631ce86c6bbc76335fd0c57f2a9d3aec78e52e4e3157cc
NONCE:  13000

* 마이닝이 안되는 경우
    * 발생한 트랜잭션이 처리되지 않고 여러 건이 적체되어 있는 경우
    * 처리비용이 낮은 경우
    * 트랜잭션 nonce 앞 번호가 처리 되지 않았는데 뒤 번호가 대기하고 있는 경우, 뒤 트랜잭션은 대기하게 된다.

* 마이닝이 적체되어 안 지워지는 경우
    - geth를 종료
    - datadir 아래 geth디렉토리 밑의 transactions.rlp를 삭제
    - geth 다시 시작한다.

따라서 마이닝을 할 경우, ```txpool.inspect``` 명령어를 필요할 경우 실행하여 적체되어 있는 거래가 있는지 확인하는 것이 바람직하다.

In [ ]:
!geth --exec txpool.inspect attach http://117.16.44.45:8445

## 5.7 gas

트랜잭션을 처리하기 위해서는 비용이 발생한다. 노드들의 컴퓨팅 자원이 투입되기 때문에 이에 대한 비용이다.
비용은 송금 금액에 비례하는 것이 아니라, 데이터 크기와 소요되는 컴퓨팅 자원량에 따라 산정이 된다.
비용은 소모되는 **gas**에 **gasPrice** 단가를 곱하여 계산한다.
주유소에 비유해서 이해할 수 있다.
주행거리에 따라 필요한 연료량과 단가에 따라 주유를 하는 방식이다.
즉 **gas는 필요한 리터**에 해당한다. 예를 들어 Kecchak hashing을 하는데 30 단위의 gas가 필요, 일정 분량의 바이트를 실행하는데 몇 gas가 필요한지 나타내는 단위를 의미한다.

이러한 거래에 비용이 발생하게 되면, DDos 공격 역시 상당한 비용이 수반되게 되므로 쉽게 시도하지 못하게 되는 효과가 있다.
블록체인에 부담이 가는 실행 (무한반복...) 역시 gas비용때문에 가능하지 못하게 된다. 일정 gas가 되면 gas limit을 초과하게 되므로 정지하게 된다.

```python
거래 비용 = gas price * gas
```

### gasPrice

gasPrice는 단가로서 사용자가 정할 수 있다.
주유와 비유하면 gas price는 시가, 즉 몇 ether가 필요한지에 해당한다.
보통 명시하지 않으면 최선의 가격으로 정해지게 된다.
gasPrice에 따라 처리속도가 결정된다.
gasPrice가 너무 적으면 마이닝되지 않을 수 있으며, 반대로 많으면 빠르게 마이닝 될 수 있다.
gasPrice가 높을수록 더 많은 마이너가 처리하려고 들 것이고 트랜잭션이 마이닝되는 시간이 짧아지게 된다.
예를 들어, 2020년 현재 5 Gwei ($10 \times 10^9$)로 정하면 2분 이내로 빠르게 처리된다. https://ethgasstation.info/

Speed | Gas Price (gwei)
-----|-----
SafeLow (<30m) | 1
Standard (<5m) | 1.3
Fast (<2m) | 5

메인네트워크에서 gasPrice 단가는 시간에 따라 정해져 있지 않고 변동한다
etherscan.io에서는 지난 수 년간의 gasPrice를 그래프로 제공하고 있는데 (https://etherscan.io/chart/gasprice), 그 가격이 약간씩 감소하고 있지만 이는 이더리움의 실물가치가 상대적으로 증가하면서 생기는 현상일 것이다.

gas price는 ```web3.eth.getGasPrice()``` 함수를 사용하여 구할 수 있고, 개인망에서 그 값은 1 gwei를 출력하고 있다.
보통 테스트네트워크는 마이닝의 우선순위에 영향을 미치지 않기 때문에, 고정 gas price를 가지고 있다.

gasPrice()를 계산하기 위해서는
```python
> web3.eth.getGasPrice().then(console.log);
> 1000000000
```

gasPrice변수에 getGasPrice()의 결과를 입력하기 위해서는:
```python
> web3.eth.getGasPrice().then(function(gp) {console.log(gp); gasPrice=gp;});
> 1000000000
```

```python
> web3.eth.getGasPrice().
... then(function(gp) {
...     console.log("Average gas price: " + gp);
...     gasPrice = gp;
... }).catch(console.error);
> Average gas price: 1000000000
```

### gas limit

gas limit은 거래의 처리에 필요한 사용 한도를 말한다. 사용자가 최대로 지불할 용의가 있는 한도를 말한다.
gas limit은 거래의 성격에 따라 다르게 책정된다. 최소 21,000에서, 단순한 거래의 경우 보통 30,000 정도면 충분하다.
복잡한 거래일수록 필요한 gas가 늘어난다.
gas가 소진될 때 까지 마이닝하고, 남으면 반환, 모자라면 **Out of Gas Exception**이 발생한다.

또한 gas limit은 블록에 대한 한도를 의미하며, 이는 블록에 포함될 거래의 갯수를 결정하기 위해 사용된다.
예를 들어, 거래가 6개 있고 gas비가 각 25, 30, 35, 40, 45, 50이라고 하자. 블록의 거래한도는 100이라고 하자.
그러면 거래의 gas가 25 + 30 + 35를 포함할 수도, 40+45가 포함될 수도 있다.

지금 현재 9,994,671 gas이고 (https://ethstats.net/) 이를 21000(거래 gas limit)으로 나누면 475개의 거래가 한 블록에 포함되는 수준이다.
이 속도로 평균 14초 내외로 1블록이 생성되고 있다 (https://ethstats.net/).
블록에 대한 gas limit은 처음에 ```genesis.json```에 설정되어 있고, 블록마다 이 값은 마이너들에 의해 1/1024 만큼씩 조정될 수 있다.
블록을 생성하면서 그만큼의 한도로 조정은 할 수 있으나, 큰 차이로 변경하는 것은 허용되지 않는다.
```eth.getBlock('latest').gasLimit```은 현재의 gasLimit을 알 수 있다.

In [5]:
!geth --exec "eth.getBlock('latest').gasLimit" attach http://117.16.44.45:8445

8000000


### gas 산정

**실행비용 execution costs**는 사용되는 명령어 opcode를 기준으로 산정이 된다.

코드 | 명령어 | 비용 | 설명
-----|-----|-----|-----
0x00 | stop | | 실행중지
0 01 | ADD | 3 | 더하기
10 | LT | 3 | less than 비교
20 | SHA3 | 30 | Keccak-256 hash 계산
30 | ADDRESS |  | 현재 실행 계정 주소 읽기
40 | BLOCKHASH | 20 | 블록 해시


#### Opcodes

기계어 수준의 명령코드, **Op**eration **Codes**를 말한다 (참조 https://ethervm.io/)
예를 들어, 스택에 ```2 3 ADD```라고 저장이 되어 있고 (push), 하나씩 꺼내 (pull) ```2 + 3``` 연산을 한다.

* ```2 3 OP_ADD 5 OP_EQUAL``` $2+3==5$
* ```2 3 OP_ADD 2 OP_MUL 1 OP_ADD 11 OP_EQUAL``` $((2+3) \times 2)+1==1$
* 기본적인 2개의 스크립트
    * scriptPubKey: locking script, pubKey로 잠그는 명령문
    DUP HASH160 <PubHash> EQAULVERIFY CHECKSIG
    * scriptSig: unlocking script, pubKey를 푸는 sig로 해제하는 명령문. 
    <sig> <PubK>
* P2PKH Pay-to-Public-Key-Hash script
    * scriptSig, scriptPubKey를 합쳐서 실행
    <서명> <공개키> OP_DUP OP_HASH160 <공개키Hash> OP_EQUALVERIFY OP_CHECKSIG
* Pay-to-Public-Key
     <Signature from Private Key A> <Public Key A> OP_CHECKSIG

**거래 비용 transaction costs**는
* 보통 거래의 처리에는 21,000, 컨트랙 배포에는 32,000 gas가 소요
* byte수에 따라 산정된다. Yellow Paper Appendix G를 참조해서 바이트수에 따른 비용을 알 수 있다. 데이터 값이 없는 $G_{txdatazero}$는 4, 데이터 값이 있는 $G_{txdatanonzero}$는 68로 계산된다. $1594 \times 68 + 184 \times 4 = 109128$로 계산한다.


![alt text](figures/5_gasCost.png "gas cost in yellow paper")

아래와 같은 트랜잭션 데이터가 있다고 하자.

In [3]:
txdata = "606060405260405160808061067283398101604090815281516020830151918301516060909301519092905b42811161003457fe5b60008054600181016100468382610100565b916000526020600020900160005b8154600160a060020a038089166101009390930a928302920219161790555060008054600181016100858382610100565b916000526020600020900160005b8154600160a060020a038088166101009390930a928302920219161790555060008054600181016100c48382610100565b916000526020600020900160005b8154600160a060020a038087166101009390930a928302920219161790555060028190555b5050505061014b565b8154818355818115116101245760008381526020902061012491810190830161012a565b5b505050565b61014891905b808211156101445760008155600101610130565b5090565b90565b6105188061015a6000396000f300606060405236156100a15763ffffffff7c0100000000000000000000000000000000000000000000000000000000600035041663120b205381146100a35780634ecb35c4146100c557806366d003ac146100f557806367e404ce146101215780636cc6cde11461014d5780637022b58e1461017957806393c0e83a1461018b578063a79a3cee146101ba578063ac4c25b2146101de578063c7fe23a3146101f0575bfe5b34156100ab57fe5b6100b3610220565b60408051918252519081900360200190f35b34156100cd57fe5b6100e1600160a060020a0360043516610226565b604080519115158252519081900360200190f35b34156100fd57fe5b61010561023b565b60408051600160a060020a039092168252519081900360200190f35b341561012957fe5b610105610273565b60408051600160a060020a039092168252519081900360200190f35b341561015557fe5b6101056102ab565b60408051600160a060020a039092168252519081900360200190f35b341561018157fe5b6101896102e3565b005b341561019357fe5b610105600435610364565b60408051600160a060020a039092168252519081900360200190f35b34156101c257fe5b6100e1610396565b604080519115158252519081900360200190f35b34156101e657fe5b610189610410565b005b34156101f857fe5b6100e1600160a060020a0360043516610479565b604080519115158252519081900360200190f35b60025481565b60016020526000908152604090205460ff1681565b60006000600181548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b60006000600081548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b60006000600281548110151561024d57fe5b906000526020600020900160005b9054906101000a9004600160a060020a031690505b90565b6102ec33610479565b15156102f85760006000fd5b600160a060020a0333166000908152600160208190526040909120805460ff19169091179055610326610396565b1561035f5761033361023b565b604051600160a060020a039182169130163190600081818185876187965a03f192505050151561035f57fe5b5b5b5b565b600080548290811061037257fe5b906000526020600020900160005b915054906101000a9004600160a060020a031681565b600080805b60005481101561040357600160006000838154811015156103b857fe5b906000526020600020900160005b9054600160a060020a036101009290920a900416815260208101919091526040016000205460ff16156103fa576001909101905b5b60010161039b565b600282101592505b505090565b33600160a060020a0316610422610273565b600160a060020a0316146104365760006000fd5b600254421161044157fe5b610333610273565b604051600160a060020a039182169130163190600081818185876187965a03f192505050151561035f57fe5b5b5b565b6000805b6000548110156104e15782600160a060020a03166000828154811015156104a057fe5b906000526020600020900160005b9054906101000a9004600160a060020a0316600160a060020a031614156104d857600191506104e6565b5b60010161047d565b600091505b509190505600a165627a7a72305820b02ae8d25b238d57e8363bfc5f30351753ca38bde86d49729841a827fb4e9d660029000000000000000000000000a4be607be0a6cceff581ef35c0cdc47a7532ff540000000000000000000000007c6b393dbb1152691b02a61378bc731662cdd9f1000000000000000000000000748ed23daa18226d872b5bd4d48ff2594fd6901c00000000000000000000000000000000000000000000000000000000597fc480"

2자리씩, 즉 1바이트씩 읽어서 0인지 아닌지 그 갯수를 산정한다.

In [4]:
def count_zero_bytes(data):
  count = 0
  for i in range(0, len(data), 2):
    byte = data[i:i+2]
    if byte == "00":
      count += 1
  return count

In [5]:
def count_non_zero_bytes(data):
  return (len(data) / 2) - count_zero_bytes(data)

그 결과를 출력하면 0은 184바이트, 아닌 경우는 1594 바이트가 된다.

In [6]:
print("zero-bytes: {0}".format(count_zero_bytes(txdata)))
print("non-zero-bytes: {0}".format(count_non_zero_bytes(txdata)))

zero-bytes: 184
non-zero-bytes: 1594.0


### web3.eth.estimateGas

개스비용을 적지 않으면, ```web3.eth.estimateGas()``` 함수로 산정한 값으로 적용하게 된다.

```python
var gas = web3.eth.estimateGas({
    data: byteCode
});
console.log(gas);
```

#### 사설망의 transaction fee

거래 transaction을 송신자, 수신자, gas price, gas limit, 송금액을 넣어서 생성한다.
생성된 거래객체의 처리에 필요한 gas를 ``` web3.eth.estimateGas(transactionObject)``` 함수로 산정하여, 최대지불가능한 gas limit으로 설정한다.
그 거래비용을 gas limit x gas price로 계산하여 출력한다.

In [11]:
%%writefile src/gasTransaction.js
var sender = web3.eth.accounts[0];
var receiver = web3.eth.accounts[1];
var bal = web3.eth.getBalance(sender);
//var gasPrice = web3.eth.getGasPrice();
var gasPrice;
web3.eth.getGasPrice(function(error, gasprice){
    console.log(gasprice);
    gasPrice=gasprice;
});

var transactionObject = {
  from: sender,
  to: receiver,
  gasPrice: gasPrice,
}
var gasLimit = web3.eth.estimateGas(transactionObject);
var transactionCost = gasPrice * gasLimit;

transactionObject.gas = gasLimit;
transactionObject.value = balance - transactionCost;
console.log("gas limit: ", gasLimit);  //21000
console.log("gas price: ", gasPrice);  //1000000000 1gwei
console.log("transaction fee = gas limit x gas price = ", transactionCost);   //21,000,000,000,000

Overwriting src/gasTransaction.js


In [ ]:
Async 방식이라 실행결과가 올바르게 출력되지 않는다. geth console에서 한 줄씩 입력하여 출력하면 바르게 출력이 된다.

In [12]:
!geth --exec 'loadScript("src/getTransaction.js")' attach http://117.16.44.45:8445

false


#### eth.getTransactionReceipt의 gasUsed

트랜잭션 처리 비용은 ```eth.getTransactionReceipt(hash)```로 알 수 있다.
아래에서 보듯이gasUsed는 21000이고, 이는 보통의 거래에 소요되는 gas이다.

In [3]:
!geth --exec 'eth.getTransactionReceipt("0xd87121b8b0f84f7fa038cd7c1928ca6a222d14228125c90edc2493fdef4fb90b")' attach http://117.16.44.45:8445

{
  blockHash: "0xd2c51ae5dea10e50c915e9d7ccc6c117c2d14d0f38da936b62a1c38fd0494d26",
  blockNumber: 55169,
  contractAddress: null,
  cumulativeGasUsed: 21000,
  from: "0x2e49e21e708b7d83746ec676a4afda47f1a0d693",
  gasUsed: 21000,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0x1817a1775db945025d4a67a0bfcb633b5fed6a1fa76804d152db410c9140237d",
  to: "0xe36104ad419c719e356e86f94b5a7ca47a83f9e7",
  transactionHash: "0xd87121b8b0f84f7fa038cd7c1928ca6a222d14228125c90edc2493fdef4fb90b",
  tran

#### 실제

실제 hash를 가지고 gas 상세를 살펴보자 (참조: https://etherscan.io/tx/0xcb1e3530950cf2c43a307bcb5645ae71a12c76a60831617badd04aea3efe68aa)

* Transaction Fee: 0.000284248 Ether ($0.05) = gas used x gas price = 35531 x 8 (아래를 참조)
* Gas Limit: 136,500
* Gas Used by Transaction: 35,531 (26.03%)
* Gas Price: 0.000000008 Ether (8 Gwei) 이 수준은 'Fast' 기준


![alt text](figures/5_transactionFee.png "transaction fee in a real case")

### TransactionCount

* 주소에서 전송된 거래 건수는 ```getTransactionCount``` 명령어를 통해서 알 수 있다.
```python
getTransactionCount(fromAddress, 'pending')
```

In [4]:
!geth --exec 'eth.getTransactionCount("0x2e49e21e708b7d83746ec676a4afda47f1a0d693")' attach http://117.16.44.45:8445 

25


* 발생된 거래를 살펴보기
    * eth.getBlock(55169);
    * eth.filter('latest')

In [1]:
!geth --exec 'eth.getBlock(55169);' attach http://117.16.44.45:8445 

{
  difficulty: 6314813,
  extraData: "0xd783010504846765746887676f312e372e33856c696e7578",
  gasLimit: 4712388,
  gasUsed: 21000,
  hash: "0xd2c51ae5dea10e50c915e9d7ccc6c117c2d14d0f38da936b62a1c38fd0494d26",
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  miner: "0x2e49e21e708b7d83746ec676a4afda47f1a0d693",
  mixHash: "0x4da21d2c5696ee4d798f6445db0287a73fb2332f089ac43690d302827f52401a",
  nonce: "0x3cd421957745a4b1",
  number: 55169,
  parentHash: "0x6533c71dd0f03108ea2c657896e73253f4f60ed3c4524a2bf0009cdec35

In [2]:
!geth --exec 'eth.filter("latest");' attach http://117.16.44.45:8445 

{
  callbacks: [],
  filterId: "0xa6d9df622f0acc03602e7b3c185ea906",
  getLogsCallbacks: [],
  implementation: {
    getLogs: function(),
    newFilter: function(),
    poll: function(),
    uninstallFilter: function()
  },
  options: "latest",
  pollFilters: [],
  requestManager: {
    polls: {},
    provider: {
      newAccount: function(),
      send: function github.com/ethereum/go-ethereum/console.(*bridge).Send-fm(),
      sendAsync: function github.com/ethereum/go-ethereum/console.(*bridge).Send-fm(),
      sign: function(),
      unlockAccount: function()
    },
    timeout: null,
    poll: function(),
    reset: function(keepIsSyncing),
    send: function(data),
    sendAsync: function(data, callback),
    sendBatch: function(data, callback),
    setProvider: function(p),
    startPolling: function(data, pollId, callback, uninstall),
    stopPolling: function(pollId)
  },
  formatter: function(log),
  get: function(callback),
  stopWatching: function(),
  watch: function(callb

## 문제 5-1: 다른 계정으로 송금 거래

<a id='5-1'></a>

* 1-1 같은 기계, 다른 계정으로 송금하기
* 1-2 다른 기계, 다른 계정으로 송금하기


### 1-1 같은 기계, 다른 계정으로 송금하기

타계정으로 송금을 해보자.
송금뿐만 아니라 블록체인에 데이터를 저장하기 위해서는 비용이 수반된다.
송금 거래를 네트워크에 전송하기 위해서는 gas가 필요하고, 계정에 이를 충당할 잔고가 있어야 한다.
gas가 모자라면 거래가 처리되지 않거나, 지연될 수 있다.
잔고를 충전하기 위해서는 Ether를 구매하거나, 마이닝을 해서 충전을 해두어야 한다.

우선:
* 지급계정을 선택하고, 잔고를 확인한다. 송금액과 gas비를 충당할 금액이 있어야 한다.
* 현재 blockNumber를 확인하고, 거래가 완성된 후 blockNumber와 잔고의 변화가 있는지 살펴보자.
* 개인망이라서 자신이 거래를 발생하여 스스로 마이닝을 하자.
* eth와 web3를 섞어서 사용하는 것은 피하도록 한다.

In [7]:
!geth --exec "eth.getBalance(eth.accounts[1])" attach ethhttp://117.16.44.45:8445

Fatal: Unable to attach to remote geth: no known transport for URL scheme "ethhttp"


In [2]:
!geth --exec "eth.sendTransaction({from:eth.accounts[1], to:eth.accounts[0],value:10000})" attach http://117.16.44.45:8445 

Error: authentication needed: password or unlock
    at web3.js:3143:20
    at web3.js:6347:15
    at web3.js:5081:36
    at <anonymous>:1:1



아래와 같이 프로그램을 만들어 실행해보자. 물론 geth client에서 한 줄씩 실행해도 좋다.
송금하려면, account unlock해야 한다. geth를 시작할 때 --unlock 하거나, console에서 직접 할 수 있다.
다음과 같이 프로그램을 실행하기 전에 다음과 같이 unlock을 하자.
```python
> personal.unlockAccount(eth.accounts[2])
```

```eth.sendTransaction({from:eth.coinbase, to:eth.accounts[1],value:10000})```는 coinbase에서 2번째 계정으로 송금하는 명령어이다.
```value```에 송금액을 기입하였다.
```eth.gasPrice()``` 지난 몇 건의 블록의 gasPrice의 중위값을 알려준다.

실행하고 나면, 거래가 발생하고, 마이닝을 하고, 잔고와 bloack number가 변화하게 된다.
coinbase에는 마이닝 보상까지 더해지므로, 잔고가 송금액만큼 줄어들지는 않을 것이다.

In [15]:
!geth --exec 'loadScript("src/e_testTran.js")' attach http://117.16.44.45:8445

coinbase:  0x21c704354d07f804bab01894e8b4eb4e0eba7451
sender balance in ether:  60.000021786999979076
receiver balance in ether:  184.999843394000019444
median gas price:  1000000000
block number:  47206
tranaction count:  264
...mining start
mining done...
- new sender balance in ether:  65.000021786999969076
- new receiver balance in ether:  184.999843394000029444
- block number:  47207
- tranaction count:  265
true


![alt text](figures/2_sendTransactionToAnotherAccount.png "send transaction to another account")

![alt text](figures/2_transactionReceipt.png "transaction recepit after send transaction")

### 1-2 다른 기계, 다른 계정으로 송금하기

* 117.16.44.45에서 117.16.44.46 ("0x519775cc61e4c9b3f19b75426a7a3696a3c85035") 으로 송금
* 117.16.44.45에서 멀티노드의 117.16.44.46 계정 잔고를 알 수 있다.

In [7]:
!geth --exec 'eth.getBalance("0x519775cc61e4c9b3f19b75426a7a3696a3c85035");' attach http://117.16.44.45:8445 

10000


In [11]:
!geth --exec 'loadScript("src/e_testTran2.js")' attach http://117.16.44.45:8445

hq account balance:  20000
block number:  70296
transactionHash:  0x15b17f04f20322cc8ee14aef9a9b617ec77935f29ce313a78ad1388241607e6c
...mining start
mining done...
block number:  70297
hq account balance:  30000
true


## 연습문제 1

다음 4건의 데이터에 대해 Merkle Root 값을 계산하고 출력하세요.
중간 노드의 AB, CD의 해시도 출력하세요.
* txA = 'Hello'
* txB = 'How are you?'
* txC = 'This is Thursday'
* txD = 'Happy new Year'

## 연습문제 2

친구에게 주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.
안되면 친구의 주소를 만들고 전송한다.
잔고의 증가분을 출력하세요.
소요된 gas비용 출력하세요.

### 답

우선 genesis block를 동일하게 설정해 놓은 상대 컴퓨터의 enode를 구해서, Peer로 추가해야 한다 (참조 3장 3-3).
enode에는 IP주소, 포트번호가 포함되어 있어, 상대 컴퓨터를 노드로 추가할 수 있다.

```python
admin.addPeer("enode://719b29cc599ee5964...011155676a3cb@117.16.44.46:30446?discport=0")
```

그리고 송금거래를 발생하면 된다 (참조 5-1) (아래 실행하지 않았슴 (NotYet))

## 연습문제 3

다음 ```sayHello()```함수의 ABI 명세를 생성한다.

```python
contract Hello {
    function sayHello(bytes toWhom) pure public returns(string memory) {}
}
```

## 연습문제 4

"Let's meet in my office at 10 AM."의 거래비용 gas를 계산하시오.

## 연습문제 5

**해시는 100미만의 양수**로 정해진다고 하자.
해시 값은 별도로 해싱을 할 필요없이 십진수로 생각하자.
**NONCE는 반복회수**로만 쓰이고 무작위 수를 생성하는데 입력되지는 않는다.
암호화폐에서는 마이닝이라고 하는 작업, 즉 목표 해시를 찾아보자.
**목표해시 값이 100에 얼마나 가까운지에 따라**, 즉 90은 가깝고 10은 먼 값이다.
**몇 회만에 그 값을 찾는지 비교**해 보자.
여기서 난이도를 3회만에 찾게 되면 1이라고 하자.
3회 보다 횟수가 많이 걸리면 이 경우는 어렵다는 의미이므로 난이도를 낮추어야 한다.
반대의 경우 3회보다 적은 횟수가 필요하면 난이도를 높이게 된다.
문제는 난이도를 계산해서 조정하지는 않고, 다순하게 목표 값을 몇 회에 출력하는지만 알아보자.
- 90을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력
- 10을 목표해시로 정하고 몇 회만에 마이닝에 성공하는지 출력

### random 함수
```random()```은 0~1사이의 무작위 수를 생성한다.
이 함수에 100을 곱하면 0~100 사이의 수를 생성한다 (100은 제외).
```print```문의 ```end```는 출력을 이어서 하게 만든다.

In [15]:
from random import randint
for i in range(1,20):
    print(int(random.random()*100), end=" ")

83 18 99 61 2 74 4 51 12 63 46 64 67 72 7 70 73 49 86 

### 답

```int(random.random()*100)```로 0~99까지의 수를 생성하고 (random() 함수가 0 이상~1.0 미만 사이의 수를 생성)
목표로 하는 90 이내 (또는 10) 이내의 수가 생성이 되면 반복을 종료하게 된다.

아래 6번째 줄을 90 또는 10으로 해서 실행을 해보자.
TargetHash 한도를 90으로 하면 보통 1회, 10으로 하면 훨씬 더 많은 횟수가 필요하였다.
직접 10 이하의 수를 4번 시도해 보니 5, 22, 7, 16번 회만에 생성되었다.
그러니까 90으로 하면 난이도가 낮고, 10으로 하면 난이도가 높다는 것을 알 수 있다.